### Initial Setup

In [21]:
# Suppress all warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objects as go
from prophet import Prophet
import dash
from dash import dcc, html
from dash.dependencies import Input, Output


In [18]:
# Load datasets
products = pd.read_csv('E:\data-driven-analytics_ecommerce\data\products.csv')
customers = pd.read_csv('E:\data-driven-analytics_ecommerce\data\customers.csv')
transactions = pd.read_csv(r'E:\data-driven-analytics_ecommerce\data\transactions.csv')
marketing_campaigns = pd.read_csv('E:\data-driven-analytics_ecommerce\data\marketing_campaigns.csv')
market_trends = pd.read_csv('E:\data-driven-analytics_ecommerce\data\market_trends.csv')
competitor_data = pd.read_csv('E:\data-driven-analytics_ecommerce\data\competitor_data.csv')

In [19]:
# Convert DateTime columns to datetime format using the correct format string
transactions['DateTime'] = pd.to_datetime(transactions['DateTime'], format='%Y-%m-%d')
market_trends['DateTime'] = pd.to_datetime(market_trends['DateTime'], format='%Y-%m-%d')
competitor_data['DateTime'] = pd.to_datetime(competitor_data['DateTime'], format='%Y-%m-%d')
products['Launch_Date'] = pd.to_datetime(products['Launch_Date'], format='%Y-%m-%d')
customers['Last_Purchase_Date'] = pd.to_datetime(customers['Last_Purchase_Date'], format='%Y-%m-%d')
marketing_campaigns['Start_Date'] = pd.to_datetime(marketing_campaigns['Start_Date'], format='%Y-%m-%d')
marketing_campaigns['End_Date'] = pd.to_datetime(marketing_campaigns['End_Date'], format='%Y-%m-%d')

# Display basic info for each dataset to confirm loading and conversion
for df, name in zip([products, customers, transactions, marketing_campaigns, market_trends, competitor_data],
                    ['products', 'customers', 'transactions', 'marketing_campaigns', 'market_trends', 'competitor_data']):
    print(f"\n{name} dataset:")
    print(df.info())



products dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Product_ID          5000 non-null   object        
 1   Category            5000 non-null   object        
 2   Base_Price          5000 non-null   float64       
 3   Cost_Price          5000 non-null   float64       
 4   Seasonal_Impact     5000 non-null   float64       
 5   Storage_Cost        5000 non-null   float64       
 6   Shipping_Cost       5000 non-null   float64       
 7   Warranty_Cost       5000 non-null   float64       
 8   Supplier_ID         5000 non-null   object        
 9   Supplier_Lead_Time  5000 non-null   int64         
 10  Reorder_Point       5000 non-null   int64         
 11  Min_Order_Quantity  5000 non-null   int64         
 12  Stock_Level         5000 non-null   int64         
 13  Rating              5000 non-

#### Data Preprocessing

In [24]:
# Handle missing values
products['Base_Price'] = products['Base_Price'].fillna(products['Base_Price'].median())
products['Cost_Price'] = products['Cost_Price'].fillna(products['Cost_Price'].median())
transactions['Discount_Offered'] = transactions['Discount_Offered'].fillna(0)
customers['Loyalty_Score'] = customers['Loyalty_Score'].fillna(customers['Loyalty_Score'].mean())

# Aggregate competitor data to get average competitor price per product and date
competitor_avg = competitor_data.groupby(['Product_ID', 'DateTime'])['Competitor_Final_Price'].mean().reset_index()
competitor_avg.rename(columns={'Competitor_Final_Price': 'Avg_Competitor_Price'}, inplace=True)

#### Dynamic Pricing Optimization:
Objective: Adjust product prices in real-time to maximize profit margins using historical sales, competitor pricing, and market trends.

In [25]:
# Aggregate transactions to daily level per product
daily_sales = transactions.groupby(['Product_ID', 'DateTime']).agg({
    'Final_Price': 'mean',
    'Quantity_Purchased': 'sum',
    'Discount_Offered': 'mean'
}).reset_index()

# Merge with products, competitor data, and market trends
df_pricing = pd.merge(daily_sales, products[['Product_ID', 'Category', 'Cost_Price', 'Seasonal_Impact']], 
                      on='Product_ID', how='left')
df_pricing = pd.merge(df_pricing, competitor_avg, on=['Product_ID', 'DateTime'], how='left')
df_pricing = pd.merge(df_pricing, market_trends[['DateTime', 'Google_Trend_Score', 'Market_Sentiment']], 
                      on='DateTime', how='left')

# Feature engineering
df_pricing['Price_Differential'] = (df_pricing['Final_Price'] - df_pricing['Avg_Competitor_Price']) / df_pricing['Avg_Competitor_Price']
df_pricing['Profit_Margin'] = (df_pricing['Final_Price'] - df_pricing['Cost_Price']) / df_pricing['Final_Price']
df_pricing['lag1_quantity'] = df_pricing.groupby('Product_ID')['Quantity_Purchased'].shift(1)
df_pricing['lag1_price'] = df_pricing.groupby('Product_ID')['Final_Price'].shift(1)
df_pricing['day_of_week'] = df_pricing['DateTime'].dt.weekday
df_pricing['month'] = df_pricing['DateTime'].dt.month

# Handle missing values after feature engineering
df_pricing = df_pricing.fillna({'Avg_Competitor_Price': df_pricing['Avg_Competitor_Price'].mean(),
                                'Price_Differential': 0, 'lag1_quantity': 0, 'lag1_price': df_pricing['Final_Price']})

In [26]:
# Select a sample product
sample_product = df_pricing['Product_ID'].value_counts().index[0]
df_product = df_pricing[df_pricing['Product_ID'] == sample_product].set_index('DateTime')

# Reindex to include all days and fill missing data
all_dates = pd.date_range(start=df_product.index.min(), end=df_product.index.max(), freq='D')
df_product = df_product.reindex(all_dates).fillna({'Quantity_Purchased': 0}).ffill()

# Define features and target
features = ['Final_Price', 'lag1_quantity', 'lag1_price', 'day_of_week', 'month', 
            'Avg_Competitor_Price', 'Google_Trend_Score', 'Market_Sentiment']
target = 'Quantity_Purchased'
df_product = df_product.dropna(subset=features)

# Split data
train_size = int(len(df_product) * 0.8)
train = df_product.iloc[:train_size]
test = df_product.iloc[train_size:]
X_train, y_train = train[features], train[target]
X_test, y_test = test[features], test[target]

# Train model
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Demand Prediction MSE for Product {sample_product}: {mse}")

# Price optimization function
def optimize_price(model, features_df, cost_price, price_range):
    prices = np.linspace(price_range[0], price_range[1], 100)
    profits = []
    for price in prices:
        sim_data = features_df.copy()
        sim_data['Final_Price'] = price
        demand = model.predict(sim_data[features])[0]
        profit = (price - cost_price) * demand
        profits.append(profit)
    optimal_price = prices[np.argmax(profits)]
    return optimal_price, max(profits)

# Example optimization for the last test day
last_day = X_test.tail(1)
cost_price = df_product['Cost_Price'].iloc[0]
optimal_price, max_profit = optimize_price(model, last_day, cost_price, (cost_price * 1.1, cost_price * 2))
print(f"Optimal Price: {optimal_price}, Expected Profit: {max_profit}")

Demand Prediction MSE for Product 5a24a65c-8fc9-4363-81b7-df07954f367f: 0.2805523872963895
Optimal Price: 914.8964545454545, Expected Profit: 6.2597862856695246
